In [25]:
import tensorflow as tf
import pickle
import numpy as np
import time
import matplotlib.pyplot as plt
from load_cifar import *

In [26]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [27]:
#learning rate
lr = 0.01
#number of traning steps
num_epochs =20

#number of batch_size
batch_size = 128

#network parameters
num_classes = 10

<h1>Define Placeholder</h1>

In [28]:
X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name='X')
Y = tf.placeholder(tf.float32, shape=[None, num_classes], name='Y')
drop_prob = tf.placeholder(tf.float32, name='drop_prob')
learning_rate = tf.placeholder(tf.float32, shape=[])

<h1>Define Neural Network Architecture</h1>

In [29]:
def initalize_weights(shape):
    return tf.Variable(tf.truncated_normal(
        shape = shape, 
        mean = 0, 
        stddev = 0.01))
def initialize_bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

def fully_connected_net(x, drop_prob):
    x = tf.reshape(x,[-1, 3072])    
    
    W_fc1 = initalize_weights([3072, 4000])
    b_fc1 = initialize_bias([4000])
    h_fc1 = tf.nn.relu(tf.add(tf.matmul(x, W_fc1), b_fc1))
    h_fc1_drop = tf.nn.dropout(h_fc1, rate = drop_prob)
    mean, var = tf.nn.moments(h_fc1_drop, [0, 1])
    h_bn1 = tf.nn.batch_normalization(h_fc1_drop, mean ,var, 0, 1, 0)
    
    W_fc2 = initalize_weights([4000, 1200])
    b_fc2 = initialize_bias([1200])
    h_fc2 = tf.nn.relu(tf.add(tf.matmul(h_bn1, W_fc2), b_fc2))
    h_fc2_drop = tf.nn.dropout(h_fc2, rate = drop_prob)
    mean, var = tf.nn.moments(h_fc2_drop, [0, 1])
    h_bn2 = tf.nn.batch_normalization(h_fc2_drop, mean ,var, 0, 1, 0)

    W_fc3 = initalize_weights([1200, 10])
    b_fc3 = initialize_bias([10])
    h_fc3 = tf.nn.relu(tf.add(tf.matmul(h_bn2, W_fc3), b_fc3))
    h_fc3_drop = tf.nn.dropout(h_fc3, rate = drop_prob)
    mean, var = tf.nn.moments(h_fc3_drop, [0, 1])
    logits = tf.nn.batch_normalization(h_fc3_drop, mean ,var, 0, 1, 0)

#     W_fc4 = initalize_weights([100, 10])
#     b_fc4 = initialize_bias([10])
#     logits = tf.nn.softmax(tf.add(tf.matmul(h_bn3, W_fc4), b_fc4))
    return logits


<h1>Define cost andoptimization</h1>

In [30]:
#predicted labels
logits = fully_connected_net(X,drop_prob)
#define loss

# Loss and Optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train_optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.01).minimize(loss)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [31]:
X_valid, Y_valid = pickle.load(open('cifar-preprocessed/batch_valid','rb'))
test_features, test_labels = pickle.load(open('cifar-preprocessed/batch_test','rb'))

In [32]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    #Initialize the variables
    sess.run(init)
    #fetch batch
    for i in range(num_epochs):
        start = time.time()
        for batch_id in np.arange(1,6):
            for fearues_batch, labels_batch in load_preprocessed_training_batch(batch_id,batch_size):
                #run optimization
                sess.run(train_optimizer, feed_dict={X:fearues_batch, 
                                                     Y:labels_batch,
                                                     learning_rate : lr,
                                                         drop_prob: 0.3})
        end= time.time()
        print("step "+str(i)+", Train Time = {:.3f}".format(end-start))
#         acc = sess.run(accuracy,feed_dict={X:fearues_batch,
#                                            Y:labels_batch,
#                                            learning_rate : lr,
#                                           drop_prob : .0})
#         print("    Train Accuracy= {:.3f}".format(acc))
        acc = sess.run(accuracy,feed_dict={X:X_valid,
                                           Y:Y_valid,
                                           learning_rate : lr,
                                          drop_prob : 0.0})
        print("    Validation Accuracy= {:.3f}".format(acc))
    print("Training finished!") 
    print("Testing ACcuracy:", sess.run(accuracy, 
                                        feed_dict={X:test_features, 
                                                   Y:test_labels,
                                                   learning_rate : lr,
                                                  drop_prob:0.0}))

step 0, Train Time = 6.285
    Validation Accuracy= 0.368
step 1, Train Time = 6.070
    Validation Accuracy= 0.386
step 2, Train Time = 5.971
    Validation Accuracy= 0.425
step 3, Train Time = 5.965
    Validation Accuracy= 0.443
step 4, Train Time = 5.967
    Validation Accuracy= 0.425
step 5, Train Time = 6.011
    Validation Accuracy= 0.447
step 6, Train Time = 6.056
    Validation Accuracy= 0.453
step 7, Train Time = 5.965
    Validation Accuracy= 0.464
step 8, Train Time = 5.963
    Validation Accuracy= 0.451
step 9, Train Time = 5.968
    Validation Accuracy= 0.450
step 10, Train Time = 5.976
    Validation Accuracy= 0.465
step 11, Train Time = 6.041
    Validation Accuracy= 0.493
step 12, Train Time = 5.975
    Validation Accuracy= 0.473
step 13, Train Time = 5.956
    Validation Accuracy= 0.469
step 14, Train Time = 5.964
    Validation Accuracy= 0.498
step 15, Train Time = 5.974
    Validation Accuracy= 0.496
step 16, Train Time = 5.958
    Validation Accuracy= 0.478
step 17

Under the Same Condition:

Best Accuracy W/O batch normalization: **0.40**

Best Accuracy with batch size 256: **0.45**

Best Accuracy with dropout prob 0.5: **0.4255**

Best Accuracy with Adam Optimizer: **0.41**

Best Accuracy with 10 epochs: **0.47**